In [2]:
from __future__ import print_function
import numpy as np 
np.random.seed(1337) 

In [3]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D
from keras.layers import Conv2D

Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [4]:
batch_size = 32
num_classes = 10
epochs = 5

In [5]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Redidual Connections 

Take a look at the residual connection figure here: https://arxiv.org/pdf/1512.03385.pdf

In [7]:
# input layer is the same as our typical CNN model 
inputs = Input(shape=(32, 32, 3))

## ----------- New Stuff Starts Here --------- 

# 3x3 conv with 3 output channels (same as input channels)
# Necessary that we keep the number of channels the same so that the layers can be added 

y = Conv2D(3, (3, 3), padding='same')(inputs)

# This is the residual connection link. Notice that we are no longer sequential 
z = keras.layers.add([inputs, y])   # this returns x + y.


## ----------- New Stuff Ends Here --------- 

# Rest of the model, again, remains the same 

x = Conv2D(8, (3, 3))(z)    # Notice the 'z'  
x = Activation('relu')(x) 
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Flatten()(x) 

x = Dense(num_classes)(x) 

output = Activation('softmax')(x) 



In [8]:
model = Model([inputs], output)

In [9]:
model.summary()  # Notice the 'Connected To' in the summary 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 3)    84          input_1[0][0]                    
__________________________________________________________________________________________________
add_1 (Add)                     (None, 32, 32, 3)    0           input_1[0][0]                    
                                                                 conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 30, 30, 8)    224         add_1[0][0]                      
__________

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 94s 2ms/step - loss: 2.0192 - acc: 0.2810 - val_loss: 1.8421 - val_acc: 0.3553
Epoch 2/5
38304/50000 [=====================>........] - ETA: 20s - loss: 1.7651 - acc: 0.3900